<a href="https://colab.research.google.com/github/moaaztaha/Image-Captioning/blob/main/skipgram_Ar_pretrained_with_Arabert_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jul 17 13:20:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# setting up kaggle json
!mkdir /root/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# downloading dataset from kaggle
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 98% 1.02G/1.04G [00:27<00:00, 42.2MB/s]
100% 1.04G/1.04G [00:27<00:00, 40.3MB/s]


In [3]:
# get the code form github
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 782, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 782 (delta 0), reused 3 (delta 0), pack-reused 778
Receiving objects: 100% (782/782), 62.41 MiB | 16.66 MiB/s, done.
Resolving deltas: 100% (471/471), done.
Checking out files: 100% (142/142), done.


In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from torch.utils.tensorboard import SummaryWriter
from os import path as osp

In [6]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 300  # dimension of word embeddings
attention_dim = 300  # dimension of attention linear layers
decoder_dim = 300  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 30  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
pretrained_embeddings = True
fine_tune_embeddings = True
checkpoint = None  # path to checkpoint, None if none

In [11]:
DATA_NAME = 'flickr8k_skipgram_ar_arabert_pretrained'

# local
# DATA_JSON_PATH = 'ar_data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = '/content/Image-Captioning/notebooks/ar_data.json'
IMGS_PATH = 'flickr8k/images/'

In [12]:
max_seq = 65
vocab = build_vocab(DATA_JSON_PATH, max_seq=max_seq)
vocab_len = len(vocab); vocab_len

100%|██████████| 24000/24000 [00:00<00:00, 213023.27it/s]


3309

In [13]:
list(vocab.itos.keys())[:10], list(vocab.itos.values())[:10]

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 ['<pad>', '<sos>', '<eos>', '<unk>', '+', 'ة', 'طفل', 'صغير', 'تتسلق', 'إلى'])

### Pre-trained Arabic Embeddings

In [18]:
# downloading arabic skipgrams pretrained word embedings
! wget https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_300_wiki.zip
! unzip -q full_grams_sg_300_wiki.zip

In [19]:
import gensim
model = gensim.models.Word2Vec.load("./full_grams_sg_300_wiki.mdl")
model.wv.save_word2vec_format("aravec.txt")

In [20]:
import numpy as np
def get_weights(embedding_path):
    embeddings_index = {}
    with open(embedding_path) as f:
        for line in f:
            word, coefs = line.split(maxsplit=1)
            coefs = np.fromstring(coefs, "f", sep=" ")
            embeddings_index[word] = coefs
    print("Found %s word vectors." % len(dict(embeddings_index)))
    
    num_tokens = len(vocab)
    embedding_dim = 300
    hits = 0
    misses = 0
    embedding_matrix = np.zeros((num_tokens, embedding_dim))
    for word, index in tqdm(vocab.stoi.items()):
        if word in embeddings_index:
            embedding_matrix[index] = embeddings_index[word]
            hits+=1
        else:
            misses+=1
            embedding_matrix[index] = np.random.uniform(-.1, .1, size=embedding_dim)
    print("Hist:", hits, " | Misses:", misses)
    return embedding_matrix

In [21]:
embedding_matrix = get_weights("./aravec.txt")

Found 662110 word vectors.


100%|██████████| 3309/3309 [00:00<00:00, 238020.10it/s]

Hist: 2723  | Misses: 586


In [22]:
embedding_matrix.shape

(3309, 300)

In [23]:
len(vocab.itos)

3309

In [24]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
    'fine_tune_embeddings': fine_tune_embeddings,
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout,
    'embeddings_matrix': embedding_matrix
}

logger_dic = {
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
    'max_seq_length': max_seq,
    'vocab_size': vocab_len,
    'enocder': 'resnet101',
    'dropout': dropout,
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim 
    
}


t_params

{'batch_size': 256,
 'data_name': 'flickr8k_skipgram_ar_arabert_pretrained',
 'decoder_lr': 0.0004,
 'df_path': '/content/Image-Captioning/notebooks/ar_data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_embeddings': True,
 'fine_tune_encoder': False,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': True,
 'vocab': <dataset.Vocabulary at 0x7f2df4637fd0>,
 'workers': 2}

In [25]:
# experiment name
name = DATA_NAME + "pretrained"
# path
log_dir = 'experiments'

logger = SummaryWriter(log_dir=osp.join(log_dir, name))

In [27]:
fit(t_params=t_params, m_params=m_params, logger=logger)

Loading Data
Dataset split: train
Unique images: 6000
Total size: 18000
Dataset split: val
Unique images: 1000
Total size: 3000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [0][0/71]	Batch Time 11.189 (11.189)	Data Load Time 4.409 (4.409)	Loss 8.9688 (8.9688)	Top-5 Accuracy 0.124 (0.124)
Epoch train time 184.974 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/12]	Batch Time 6.161 (6.161)	Loss 5.0141 (5.0141)	Top-5 Accuracy 53.330 (53.330)	
----- Bleu-n Scores -----
1: 61.150
2: 43.565
3: 21.524
4: 10.850
-------------------------

 * LOSS - 5.085, TOP-5 ACCURACY - 53.280, BLEU-4 - 10.85

Epoch validation time 35.935 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [1][0/71]	Batch Time 6.175 (6.175)	Data 

In [28]:
!ls

aravec.txt
BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained.pth.tar
checkpoint_flickr8k_skipgram_ar_arabert_pretrained.pth.tar
drive
experiments
flickr8k
flickr8kimagescaptions.zip
full_grams_sg_300_wiki.mdl
full_grams_sg_300_wiki.mdl.trainables.syn1neg.npy
full_grams_sg_300_wiki.mdl.wv.vectors.npy
full_grams_sg_300_wiki.zip
Image-Captioning
sample_data


In [29]:
m = load_checkpoint("BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained.pth.tar")

Loaded Checkpoint!!
Last Epoch: 14
Best Bleu-4: 21.851


In [31]:
batch_size = 64
fine_tune_encoder = True
checkpoint = 'BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained.pth.tar'
# epochs = 30

t_params['batch_size'] = batch_size
t_params['data_name'] = t_params['data_name'] + "_finetune" 
t_params['fine_tune_encoder'] = True
t_params['decoder_lr'] = t_params['decoder_lr'] / 10
# t_params['epochs'] = epochs
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_skipgram_ar_arabert_pretrained_finetune_finetune',
 'decoder_lr': 4.000000000000001e-06,
 'df_path': '/content/Image-Captioning/notebooks/ar_data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_embeddings': True,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': True,
 'vocab': <dataset.Vocabulary at 0x7f2df4637fd0>,
 'workers': 2}

In [32]:
fit(t_params, checkpoint=checkpoint, m_params=m_params, logger=logger)

Loaded Checkpoint!!
Starting Epoch: 15
Loading Data
Dataset split: train
Unique images: 6000
Total size: 18000
Dataset split: val
Unique images: 1000
Total size: 3000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [15][0/282]	Batch Time 4.890 (4.890)	Data Load Time 1.007 (1.007)	Loss 3.3775 (3.3775)	Top-5 Accuracy 69.990 (69.990)
Epoch: [15][100/282]	Batch Time 1.361 (1.423)	Data Load Time 0.001 (0.011)	Loss 3.5779 (3.5443)	Top-5 Accuracy 67.683 (68.539)
Epoch: [15][200/282]	Batch Time 1.380 (1.404)	Data Load Time 0.001 (0.006)	Loss 3.5456 (3.5254)	Top-5 Accuracy 68.004 (68.880)
Epoch train time 394.899 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/47]	Batch Time 1.633 (1.633)	Loss 4.3821 (4.3821)	Top-5 Accuracy 64.925 (64.925)	
----- Bleu-n Scores -----
1: 69.392
2: 50.768
3: 32.939

In [35]:
!ls

aravec.txt
BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained_finetune_finetune.pth.tar
BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained.pth.tar
checkpoint_flickr8k_skipgram_ar_arabert_pretrained_finetune_finetune.pth.tar
checkpoint_flickr8k_skipgram_ar_arabert_pretrained.pth.tar
drive
experiments
flickr8k
flickr8kimagescaptions.zip
full_grams_sg_300_wiki.mdl
full_grams_sg_300_wiki.mdl.trainables.syn1neg.npy
full_grams_sg_300_wiki.mdl.wv.vectors.npy
full_grams_sg_300_wiki.zip
Image-Captioning
sample_data


### Test Scores

In [38]:
!cp BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained_finetune_finetune.pth.tar /content/drive/MyDrive/ImageCaptioning/skipgram/

In [36]:
checkpoint = load_checkpoint("BEST_checkpoint_flickr8k_skipgram_ar_arabert_pretrained_finetune_finetune.pth.tar")
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval();

Loaded Checkpoint!!
Last Epoch: 21
Best Bleu-4: 23.536


In [37]:
from eval import test_score

test_dict = {}

for i in [1, 3, 5]:
    
    b1, b2, b3, b4 = test_score(i, encoder, decoder, IMGS_PATH, DATA_JSON_PATH, vocab)
    if i == 3:
        test_dict['b1'] = b1
        test_dict['b2'] = b2
        test_dict['b3'] = b3
    
    test_dict[f'b4-b{i}'] = b4

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Dataset split: test
Unique images: 1000
Total size: 3000


EVALUATING AT BEAM SIZE 1:   0%|          | 0/1000 [00:00<?, ?it/s]Image-Captioning/eval.py:93: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prev_word_inds = top_k_words // vocab_size  # (s)
EVALUATING AT BEAM SIZE 1: 100%|██████████| 1000/1000 [00:58<00:00, 17.01it/s]


----- Bleu-n Scores -----
1: 57.457
2: 43.794
3: 31.863
4: 22.814
-------------------------
Dataset split: test
Unique images: 1000
Total size: 3000


EVALUATING AT BEAM SIZE 3: 100%|██████████| 1000/1000 [00:58<00:00, 17.04it/s]


----- Bleu-n Scores -----
1: 59.903
2: 47.406
3: 36.130
4: 26.897
-------------------------
Dataset split: test
Unique images: 1000
Total size: 3000


EVALUATING AT BEAM SIZE 5: 100%|██████████| 1000/1000 [01:03<00:00, 15.73it/s]


----- Bleu-n Scores -----
1: 58.708
2: 46.523
3: 35.712
4: 27.012
-------------------------
